In [2]:
import litellm
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from judge import Judge
import json
import os

load_dotenv()


#load the dataset
df_100 = pd.read_csv("../output_data/100_data_math500.csv")
# Prompts
COT_PROMPT = """
You are a math tutor helping a student. Solve the following math problem step-by-step.

Question:
{question}

Let's think step-by-step.
"""

REFLECTION_PROMPT = """
Now reflect on the solution above. Was the reasoning and final answer correct? If there was a mistake, explain it and provide a corrected solution.
"""
judge = Judge(model='gemini/gemini-2.0-flash')

df_100.head(2)


,Unnamed: 0,problem,solution,answer,subject,level,unique_id
0,0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json


In [5]:
def generate_training_messages(df, start_idx:int, num_rows:int):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''You are a math tutor helping a student. Solve the following math problem step-by-step.
                  Question: <QUESTION>
                  Let's think step-by-step.
                ''',
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': f'solution : {row['solution']}' + '\n\n' + f'answer : {row['answer']}'
            }
        ])
        
    return messages

In [6]:
def metatuning(start_idx:int, num_rows:int,model,df_result):
    training_message = generate_training_messages(df_100, start_idx=start_idx, num_rows=num_rows)
    # LLM Call
    def call_litellm(model="openai/gpt-4o",messages=[]):
        response = litellm.completion(
            model=model,
            messages=messages,
            temperature=0
        )
        return response['choices'][0]['message']['content'].strip()

    temp_json_path = f"../output_data/temp_metatuning_rows.json"
    if os.path.exists(temp_json_path):
        with open(temp_json_path, 'r') as f:
            rows = json.load(f)
    else:
        rows = []
    already_done = len(rows)
    df_sliced = df_100.iloc[start_idx+num_rows+already_done:]
    index = 0
    for _,item in tqdm(df_sliced.iterrows()):
        index += 1
        print(index)
        q = item["problem"]
        truth = item["answer"]

        # Chain of Thought
        cot_prompt = COT_PROMPT.format(question=q)
        messages = training_message
        messages.append({"role": "user", "content": cot_prompt})
        cot_response = call_litellm(model=model,messages=messages)

        # Self Reflection
        reflection_prompt = cot_response + "\n\n" + REFLECTION_PROMPT
        reflection_response = call_litellm(model=model,messages=[{"role": "user", "content": reflection_prompt}])

        result = judge.prediction(query=q,answer1=reflection_response,answer2=truth).correct

        rows.append({
            "question": q,
            "ground_truth": truth,
            f"cot_answer_{num_rows}": cot_response,
            f"reflected_answer_{num_rows}": reflection_response,
            f"result_{num_rows}": result
        })
        # Save rows to temporary JSON file
        with open(temp_json_path, 'w') as f:
            json.dump(rows, f)

    # Create a DataFrame
    df_curr_result = pd.DataFrame(rows)
    #delete the temp json file
    os.remove(temp_json_path)

    #merge
    df_result = pd.merge(df_result,df_curr_result,on=["question","ground_truth"],how="left")
    return df_result

In [7]:
result_df_gpt4o = pd.read_csv("../output_data/math500_cot_reflection_output_gpt_4o.csv")
result_df_gemini = pd.read_csv("../output_data/math500_cot_reflection_output_gemini_1_5_flash.csv")

In [24]:
import time
df_result = pd.read_csv("../output_data/math500_metatuning_cot_reflection_output_gpt_4o.csv")
# df_result = metatuning(0,5,"openai/gpt-4o",result_df_gpt4o)
# df_result = metatuning(0,10,"openai/gpt-4o",df_result)
# df_result = metatuning(0,20,"openai/gpt-4o",df_result)
# df_result = metatuning(0,30,"openai/gpt-4o",df_result)
a = True
while a:
  try:
    df_result = metatuning(0,40,"openai/gpt-4o",df_result)
    a = False
  except Exception as e:
    time.sleep(61)
    a = True

0it [00:00, ?it/s]

1


1it [00:10, 10.64s/it]

2


2it [00:35, 19.11s/it]

...


0it [00:00, ?it/s]

1


1it [00:13, 13.17s/it]

2


2it [00:41, 20.87s/it]


In [25]:
df_result.to_csv("../output_data/math500_metatuning_cot_reflection_output_gpt_4o.csv",index=False)

In [28]:
import time
for val in [20,30,40]:
  print(f"********************{val}*****************")
  a = True
  while a:
    try:
      df_result = metatuning(0,val,"gemini/gemini-1.5-flash",df_result)
      a = False
      df_result.to_csv("../output_data/math500_metatuning_cot_reflection_output_gemini_1.5.csv",index=False)
    except Exception as e:
      time.sleep(61)
      a = True

********************20*****************


0it [00:00, ?it/s]

1


1it [00:06,  6.09s/it]

2


...

58it [32:25, 60.61s/it]

59


59it [33:34, 62.97s/it]

60


60it [34:15, 34.26s/it]


In [29]:
df_result.to_csv("../output_data/math500_metatuning_cot_reflection_output_gemini_1.5_flash.csv",index=False)

In [3]:
df_result_gpt4o = pd.read_csv("../output_data/math500_metatuning_cot_reflection_output_gpt_4o.csv")
df_result_gemini = pd.read_csv("../output_data/math500_metatuning_cot_reflection_output_gemini.csv")

In [4]:
df_result_gpt4o.columns

Index(['question', 'ground_truth', 'cot_answer', 'reflected_answer', 'result',
       'cot_answer_5', 'reflected_answer_5', 'result_5', 'cot_answer_10',
       'reflected_answer_10', 'result_10', 'cot_answer_20',
       'reflected_answer_20', 'result_20', 'cot_answer_30',
       'reflected_answer_30', 'result_30', 'cot_answer_40',
       'reflected_answer_40', 'result_40'],
      dtype='object')

In [7]:
# Function to analyze result pairs
def analyze_result_pair(df, base_col, compare_col):
    # Create a mask for non-blank values in both columns
    mask = df[base_col].notna() & df[compare_col].notna()
    
    # Get the filtered dataframe
    filtered_df = df[mask]
    
    print(f"\nAnalyzing {base_col} vs {compare_col}")
    print(f"Total rows after removing blanks: {len(filtered_df)}")
    
    # Get value counts for both columns
    print(f"\n{base_col} value counts:")
    print(filtered_df[base_col].value_counts())
    
    print(f"\n{compare_col} value counts:")
    print(filtered_df[compare_col].value_counts())
    
    
    return filtered_df

# Analyze each pair
result_pairs = [
    ('result', 'result_5'),
    ('result', 'result_10'),
    ('result', 'result_20'),
    ('result', 'result_30'),
    ('result', 'result_40')
]


In [8]:
for base_col, compare_col in result_pairs:
    filtered_df = analyze_result_pair(df_result_gpt4o, base_col, compare_col)
    print("-" * 50)


Analyzing result vs result_5
Total rows after removing blanks: 95

result value counts:
result
True     75
False    20
Name: count, dtype: int64

result_5 value counts:
result_5
True     79
False    16
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_10
Total rows after removing blanks: 90

result value counts:
result
True     71
False    19
Name: count, dtype: int64

result_10 value counts:
result_10
True     72
False    18
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_20
Total rows after removing blanks: 80

result value counts:
result
True     67
False    13
Name: count, dtype: int64

result_20 value counts:
result_20
True     68
False    12
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_30
Total rows after removing blanks: 70

result value counts:
result
True     59
False    11
Name: count, dtype: int64

result

In [9]:
for base_col, compare_col in result_pairs:
    filtered_df = analyze_result_pair(df_result_gemini, base_col, compare_col)
    print("-" * 50)


Analyzing result vs result_5
Total rows after removing blanks: 95

result value counts:
result
True     75
False    20
Name: count, dtype: int64

result_5 value counts:
result_5
False    62
True     33
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_10
Total rows after removing blanks: 90

result value counts:
result
True     71
False    19
Name: count, dtype: int64

result_10 value counts:
result_10
False    63
True     27
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_20
Total rows after removing blanks: 80

result value counts:
result
True     67
False    13
Name: count, dtype: int64

result_20 value counts:
result_20
False    42
True     38
Name: count, dtype: int64
--------------------------------------------------

Analyzing result vs result_30
Total rows after removing blanks: 70

result value counts:
result
True     59
False    11
Name: count, dtype: int64

result